## Stacking Blocks
This is a fun program demonstrating non blocking code with the robot arm. Start with all four blocks in stack 1. The robot will pick them up and move them to a random new stack and continue to do so until all blocks end up back in the same stack. The robot uses non blocking code to monitor the force on the end effector and detect when a block is found. If you are using your own blocks, make sure to update the block height variable. 

In [1]:
from myur import MyUR3e
import time
import random
robot = MyUR3e()

In [2]:
BLOCK_HEIGHT = 0.03 # rough estimate of block height

STACK0 = [0.15,0.35] # stack1 x,y
STACK1 = [0.25,0.35] # stack2 x,y
STACK2 = [0.15,0.25] # stack3 x,y
STACK3 = [0.25,0.25] # stack3 x,y
ACTIVE = [BLOCK_HEIGHT*5+0.1,0,0,0] # transfer height, orientation

DROP = [] # relative drop trajectory, 15cm max
for i in range(15):
    DROP.append([0,0,-0.01,0,0,0])

# Stack count, stack list, current stack
stacks_c = [4,0,0,0]
stacks = [STACK0,STACK1,STACK2,STACK3]
curr_stack = 0

In [3]:
# function to choose a next stack
def random_stack(grab=False):
    global curr_stack
    
    options = [0,1,2,3]
    if grab:
        options = [index for index,value in enumerate(stacks_c) if value > 0]
    else:
        options.remove(curr_stack)
        
    curr_stack = random.choice(options)
    
    return stacks[curr_stack]

# function to detect block
def detect():
    # close gripper and start drop
    robot.move_gripper(100,wait=False)
    gap = (4-stacks_c[curr_stack])
    if gap != 0: robot.move_global_r([[0,0,-gap*BLOCK_HEIGHT,0,0,0]],time=0.5*gap)
    robot.move_global_r(DROP,time=7,wait=False,interp="linear")

    # get comparative z force and monitor in loop
    comp_force = robot.read_force()[2]
    z_force = robot.read_force()[2]
    while abs(comp_force-z_force) < 1:
        z_force = robot.read_force()[2]
        time.sleep(0.05)

    # stop robot when object detected
    robot.stop()

# function to grab block
def grab():
    robot.move_global_r([[0,0,0.02,0,0,0]],time=0.3,wait=True)
    robot.move_gripper(0)
    robot.move_global_r([[0,0,-0.03,0,0,0]],time=0.3,wait=True)
    robot.move_gripper(100)

# function to drop block
def drop():
    robot.move_gripper(0,wait=False)
    robot.move_global_r([[0,0,0.03,0,0,0]],time=0.3,wait=True)
    robot.move_gripper(100,wait=False)

# reset pose
def reset_pos(stack):
    
    robot.move_gripper(0,wait=False)
    robot.move_global([stack+ACTIVE],time=3)
    robot.move_gripper(100)

In [ ]:
drop_stack = stacks[curr_stack]
reset_pos(STACK1)

while True:    
    # Choose where to grab a block
    grab_stack = random_stack(grab=True)
    robot.move_global([drop_stack+ACTIVE,grab_stack+ACTIVE],time=1)
    
    # Grab a block
    detect()
    grab()

    stacks_c[curr_stack] -= 1

    # Choose where to drop a block
    drop_stack = random_stack(grab=False)
    robot.move_global([grab_stack+ACTIVE,drop_stack+ACTIVE],time=1)

    # Drop a block
    detect()
    drop()

    stacks_c[curr_stack] += 1

    if 4 in stacks_c:
        print("I am done stacking")
        break